Notes:

python twitter api: https://python-twitter.readthedocs.io/en/latest/installation.html

1. pip install python-twitter (which also installs the dependencies: requests and requests oauthlib)
2. create a twitter app: https://apps.twitter.com/
    Requires creating a twitter account and jumping through a series of hoops so Twitter can say they've done their due diligence in, if not totally stopping Eastern Europeans from spamming their site with tweets engineered to maximize political entropy, at least acquiring the minimum necessary legal shielding such that they are protected from culpability.
3. Get your API's (secret) key and (secret) access token
4. Test your API's connection:

import twitter
api = twitter.Api(consumer_key=[consumer key],
                  
                  consumer_secret=[consumer secret],
                  
                  access_token_key=[access token],
                  
                  access_token_secret=[access token secret])
5. Assuming it didn't return an error, it's time to start pulling some data

In [ ]:
import twitter
api = twitter.Api(consumer_key='XXXXXX'
                 , consumer_secret='XXXXXX'
                 , access_token_key='XXXXXX'
                 , access_token_secret='XXXXXX'
                 )

# The default parameter for this call is `user_id` which requires an extra step to find for a given account.
# For the sake of completion, a site like this (http://gettwitterid.com) will do the legwork for you and give you the ID
posts = api.GetUserTimeline(screen_name='CosmossUp')
# OR as seen on http://gettwitterid.com/?user_name=CosmossUp&submit=GET+USER+ID
posts = api.GetUserTimeline(user_id=2320507177)

# This returns the last 20 status posts from the targeted profile in a list of `twitter.models.Status` objects, sorted
# in reverse-chronological order

print(posts[0])

# This can be pretty printed after realizing it has a json structure
import json
print(json.dumps(json.loads(str(posts[0])), indent=4, separators=(',', ': ')))

# But what we care about most is the text content of the tweet which can be accessed the easily
print(posts[0].text)


6. Next we need a way to repost this text to Slack. There are many Slack APIs but I prefer Slacker by virtue of it being the top search result the first time I googled, "slack python api." https://github.com/os/slacker

pip install slacker

7. You'll need another token, this time from Slack. Go to https://api.slack.com/custom-integrations/legacy-tokens and be greeted by a scary looking disclaimer that tells you that you can't be trusted with tokens and instead should use a Slack App$^{TM}$. Nah.

8. With your newly generated token, follow Slackers documentation to post a few or as many twitter data to channels and users your heart's content. With all of Twitter's accounts' posts and all your colleague's channels at your disposal, remember: with great power comes great responsibility.

In [ ]:
from slacker import Slacker

slack = Slacker(token='XXXXXX')

In [ ]:
# To post to a channel as yourself:
slack.chat.post_message(channel='#channel_name', text='text', as_user=True)

# To post to a channel as a bot:
slack.chat.post_message(channel='#channel_name'
                        , text='text'
                        , as_user=False
                        , username='SpaceBot'
                        , icon_url='<choose a 64x64 size image')

Our bot needs a schedule to check for new posts. One simple method is using the cron: 

http://www.unixgeeks.org/security/newbie/unix/cron-1.html

For cron noobs here's an extremely useful site that'll tell you when your schedule will run:

https://crontab.guru/

The script to grab the latest tweet, verify it's new against an ID stored in a text file, and push the results to Slack along with updating the stored ID, is below:

In [ ]:
# filename = update_and upload.py

import os
import twitter
from slacker import Slacker


# Initialize APIs w/ tokens
twitter = twitter.Api(consumer_key='<consumer key>'
			, consumer_secret='<consumer secret key>'
			, access_token_key='<access token>'
			, access_token_secret='<access token secret>')

slack = Slacker(token='<token>')

# Get filepath to project
cwd = os.path.dirname(os.path.realpath(__file__))

# Get latest tweet
post_history = twitter.GetUserTimeline(screen_name='<screen name>')
most_recent_post_id = post_history[0].id

# Creates a placeholder value for tweet ID if file doesn't exist
if not os.path.exists(cwd+'/id_history.txt'):
	with open(cwd+'/id_history.txt', 'w') as f:
		f.write('0')

# Post tweet to Slack if ID doesn't match stored tweet, otherwise take no action
with open(cwd+'/id_history.txt', 'r') as f:
	lines = f.readlines()
	for line in lines:
		previous_id = int(line.strip())

if most_recent_post_id != previous_id:
	with open(cwd+'/id_history.txt', 'w') as f:
		f.write('{}'.format(most_recent_post_id))
	post = post_history[0].text
	slack.chat.post_message(channel='<channel>'
				, text=post
				, as_user=False
				, username='<bot name>'
				, icon_url='<bot image>')
else:
	pass


In [ ]:
# In bash:
crontab -e

# In the cron's editor, add a command to execute the script every day at 10am:
0 10 * * * python /path/to/file/update_and_upload.py